In [2]:
print("all ok")

all ok


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_TRACING"]=os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"]=os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"]=os.getenv("LANGSMITH_PROJECT")

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
embeddings=OpenAIEmbeddings()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",convert_system_message_to_human=True)

In [5]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import MessagesPlaceholder

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)
     

In [7]:
doc=loader.load()

In [10]:
len(doc)

1

In [8]:
text_spliter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs=text_spliter.split_documents(doc)

In [9]:
len(docs)

134

In [12]:
docs[0]

Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng')

In [14]:
vector_store=Chroma(collection_name="testcollection",embedding_function=embeddings)

In [15]:
from uuid import uuid4
uuids=[str(uuid4()) for _ in range(len(docs))]

In [16]:
vector_store.add_documents(documents=docs,ids=uuids)

['aacacc5e-e438-473f-836d-eaf5a4d07e1c',
 '489e1582-da56-4045-9720-7e61cc939c9a',
 'a939a089-fed0-4215-81bb-a2f781dce584',
 'c455b210-aca4-4398-aa92-ae6623d14848',
 'e49ae61a-37b2-4b0c-b44a-600a09081baa',
 'bd575608-4fe5-4927-96d5-634f112d3661',
 '59d068e2-3ae9-4577-bfe7-7f7854672394',
 'a5250610-2f36-4863-ad32-6cf4771d039b',
 '733ab894-0652-4a20-a5a1-48b5f67dfc41',
 '6ccb34c6-d545-4320-b803-625707a058fe',
 '3614ee9c-898c-4646-837c-9c5941a2601e',
 '7fa757b0-b435-43f3-b995-592b99333260',
 'f0245754-150e-4bc1-84df-4dda256471e0',
 'a807cc25-f319-4876-ac00-1bd9ce8f9419',
 'ecbe5bfd-65a6-4b23-a2ca-fe6aa57f6a32',
 '8bdee82a-b28a-49cc-aa63-c031912ac274',
 '9cfad3e1-ed38-4b5b-b780-2cc6b209ae9a',
 'b582c6ee-2418-4b72-bd0d-fe02a974625b',
 '52a0e0a2-4302-4d8f-a5c6-39affbfb5ea0',
 '6b5b40c6-9343-4883-b3c5-d3af491ea1a8',
 'fc4a5f52-3717-4047-bf1a-564fff7d10eb',
 '422444cf-aeff-4976-ba2e-c7db7225838c',
 '82d4508e-2fa7-4078-9942-5664cc87b94e',
 'b8f7ade0-b91d-4b33-99f0-8ecd24cf28b4',
 '11bffe6f-e472-

In [17]:
retriever=vector_store.as_retriever(search_kwargs={"k":5})

In [18]:
system_prompt="""
You are an helpful assistant for question-answering task.
Use the following retrieve context to answer the user question.
Use maximum 3-4 sentences for your response and response should be consise and meaningful.
If you do not find the response then simply tell that you don't know.

Context:
{context}
"""

In [19]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [20]:
question_answering_chain=create_stuff_documents_chain(llm,prompt)

In [21]:
rag_chain=create_retrieval_chain(retriever,question_answering_chain)

In [22]:
response=rag_chain.invoke({"input":"what is MRKL"})

c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [24]:
response['answer']

'MRKL, which stands for “Modular Reasoning, Knowledge and Language,” is a neuro-symbolic architecture designed for autonomous agents. It comprises a collection of "expert" modules, where a general-purpose Large Language Model (LLM) functions as a router to direct inquiries to the most suitable expert. These modules can be either neural, like deep learning models, or symbolic, such as math calculators, currency converters, or weather APIs.'

In [26]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [27]:
lcel_chain=(
    {"context":retriever,"input":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
res=lcel_chain.invoke("what is MRKL")
res

c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'MRKL, short for “Modular Reasoning, Knowledge and Language,” is a neuro-symbolic architecture designed for autonomous agents. It combines a general-purpose Large Language Model (LLM) acting as a router with a collection of "expert" modules. These modules can be either neural, like deep learning models, or symbolic, such as math calculators or weather APIs, allowing the system to route inquiries to the most suitable expert.'

In [29]:
from langchain.chains import create_history_aware_retriever

In [30]:
retriever_prompt=(
    """
Given a chat history and the latest user question which might reference context in the chat
history.
formulate a standalone question which can be understood without the chat history.
Do not answer the question, just formulate it if needed and otherwise return it as is

"""
)

In [31]:
contextualized_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}")
    ]
)

In [32]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualized_prompt)

In [33]:
qa_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [34]:
question_answering_chain=create_stuff_documents_chain(llm,qa_prompt)

In [35]:
rag_chain=create_retrieval_chain(history_aware_retriever,question_answering_chain)

In [36]:
from langchain_core.messages import HumanMessage,AIMessage

In [37]:
chat_history=[]
question_1="What is task decomposition?"

In [38]:
message1=rag_chain.invoke({"input": question_1,"chat_history":chat_history})

c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [39]:
message1["answer"]

'Task decomposition is a standard technique used to break down complex tasks into smaller, simpler, and more manageable steps. This process enhances model performance by allowing the model to "think step by step" and efficiently handle intricate problems. It can be achieved through simple LLM prompting, task-specific instructions, or human input.'

In [40]:
chat_history.extend(
    [
        HumanMessage(content=question_1),
        AIMessage(content=message1['answer'])
    ]
)

In [41]:
chat_history

[HumanMessage(content='What is task decomposition?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Task decomposition is a standard technique used to break down complex tasks into smaller, simpler, and more manageable steps. This process enhances model performance by allowing the model to "think step by step" and efficiently handle intricate problems. It can be achieved through simple LLM prompting, task-specific instructions, or human input.', additional_kwargs={}, response_metadata={})]

In [42]:
second_question="what are the common ways of doing it?"
message2=rag_chain.invoke({"input":second_question,"chat_history":chat_history})
print(message2["answer"])

c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Task decomposition can be commonly done in three ways. First, by using Large Language Models (LLMs) with simple prompts such as "Steps for XYZ" or "What are the subgoals for achieving XYZ?". Second, through task-specific instructions, like "Write a story outline" for a novel. Lastly, it can also be achieved with human inputs.


In [43]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [44]:
store={}

In [45]:
def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [46]:
conversational_rag_chain=RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [47]:
conversational_rag_chain.invoke(
    {"input": "What is task decomposition?"},
    config={
        "configurable":{"session_id":"abc123"}
    }
)["answer"]

c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Task decomposition is a technique where large, complex tasks are broken down into smaller, more manageable subgoals or steps. This process enhances model performance on difficult tasks by allowing the model to "think step by step." It can be achieved through simple LLM prompting, task-specific instructions, or human input, making complex tasks more efficient to handle.'

In [48]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is task decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is a technique where large, complex tasks are broken down into smaller, more manageable subgoals or steps. This process enhances model performance on difficult tasks by allowing the model to "think step by step." It can be achieved through simple LLM prompting, task-specific instructions, or human input, making complex tasks more efficient to handle.', additional_kwargs={}, response_metadata={})])}

In [49]:
conversational_rag_chain.invoke(
    {"input": "What are the common ways of doing it?"},
    config={
        "configurable":{"session_id":"abc123"}
    }
)["answer"]

c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Task decomposition can be done in a few common ways. One method involves using Large Language Models (LLMs) with simple prompting, such as asking for "Steps for XYZ" or "subgoals." Another approach uses task-specific instructions, like "Write a story outline" for a novel. Additionally, human input can also be utilized to break down complex tasks into smaller steps.'

In [50]:
for message in store["abc123"].messages:
    if isinstance(message,AIMessage):
        prefix="AI"
    else:
        prefix="User"
    print(f"{prefix}: {message.content}")

User: What is task decomposition?
AI: Task decomposition is a technique where large, complex tasks are broken down into smaller, more manageable subgoals or steps. This process enhances model performance on difficult tasks by allowing the model to "think step by step." It can be achieved through simple LLM prompting, task-specific instructions, or human input, making complex tasks more efficient to handle.
User: What are the common ways of doing it?
AI: Task decomposition can be done in a few common ways. One method involves using Large Language Models (LLMs) with simple prompting, such as asking for "Steps for XYZ" or "subgoals." Another approach uses task-specific instructions, like "Write a story outline" for a novel. Additionally, human input can also be utilized to break down complex tasks into smaller steps.


In [51]:
conversational_rag_chain.invoke(
    {"input": "What is a prompt technique like step xyz?"},
    config={
        "configurable":{"session_id":"abc123"}
    }
)["answer"]

c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'"Steps for XYZ" is an example of a simple prompting technique used with Large Language Models (LLMs) for task decomposition. By providing such a prompt, the LLM is instructed to break down a complex task (represented by "XYZ") into a series of smaller, sequential steps, making the overall task more manageable.'

In [52]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is task decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is a technique where large, complex tasks are broken down into smaller, more manageable subgoals or steps. This process enhances model performance on difficult tasks by allowing the model to "think step by step." It can be achieved through simple LLM prompting, task-specific instructions, or human input, making complex tasks more efficient to handle.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are the common ways of doing it?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition can be done in a few common ways. One method involves using Large Language Models (LLMs) with simple prompting, such as asking for "Steps for XYZ" or "subgoals." Another approach uses task-specific instructions, like "Write a story outline" for a novel. Additionally, huma

In [53]:
for message in store["abc123"].messages:
    if isinstance(message,AIMessage):
        prefix="AI"
    else:
        prefix="User"
    print(f"{prefix}: {message.content}")

User: What is task decomposition?
AI: Task decomposition is a technique where large, complex tasks are broken down into smaller, more manageable subgoals or steps. This process enhances model performance on difficult tasks by allowing the model to "think step by step." It can be achieved through simple LLM prompting, task-specific instructions, or human input, making complex tasks more efficient to handle.
User: What are the common ways of doing it?
AI: Task decomposition can be done in a few common ways. One method involves using Large Language Models (LLMs) with simple prompting, such as asking for "Steps for XYZ" or "subgoals." Another approach uses task-specific instructions, like "Write a story outline" for a novel. Additionally, human input can also be utilized to break down complex tasks into smaller steps.
User: What is a prompt technique like step xyz?
AI: "Steps for XYZ" is an example of a simple prompting technique used with Large Language Models (LLMs) for task decompositio

In [54]:
conversational_rag_chain.invoke(
    {"input": "What is RAG system?"},
    config={
        "configurable":{"session_id":"abc123"}
    }
)["answer"]

c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\langchain_google_genai\chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'I apologize, but the provided context does not contain information about what a RAG (Retrieval Augmented Generation) system is. Therefore, I cannot answer your question based on the given text.'